In [2]:
from keras.utils import to_categorical
from tensorflow.keras.utils import load_img  
from keras.models import Sequential
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Add
from tensorflow.keras.initializers import glorot_uniform
from keras.callbacks import EarlyStopping
import os
import pandas as pd
import numpy as np

In [3]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\khang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
pip install --upgrade ipywidgets


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\khang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
TRAIN_DIR = 'C:/Users/khang/OneDrive/Desktop/Machine_Learning/images/images/train'
TEST_DIR = 'C:/Users/khang/OneDrive/Desktop/Machine_Learning/images/images/test'

In [6]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [7]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [8]:
print(train)

                                                   image     label
0      C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
1      C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
2      C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
3      C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
4      C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
...                                                  ...       ...
28816  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise
28817  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise
28818  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise
28819  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise
28820  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise

[28821 rows x 2 columns]


In [9]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [10]:
print(test)
print(test['image'])

                                                  image     label
0     C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
1     C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
2     C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
3     C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
4     C:/Users/khang/OneDrive/Desktop/Machine_Learni...     angry
...                                                 ...       ...
7061  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise
7062  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise
7063  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise
7064  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise
7065  C:/Users/khang/OneDrive/Desktop/Machine_Learni...  surprise

[7066 rows x 2 columns]
0       C:/Users/khang/OneDrive/Desktop/Machine_Learni...
1       C:/Users/khang/OneDrive/Desktop/Machine_Learni...
2       C:/Users/khang/OneDrive/Desktop/Machine_Learni...
3       C

In [11]:
from tqdm import tqdm


In [12]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [13]:
train_features = extract_features(train['image']) 

100%|██████████| 28821/28821 [00:38<00:00, 753.18it/s]


In [14]:
test_features = extract_features(test['image'])

100%|██████████| 7066/7066 [00:07<00:00, 969.73it/s] 


In [15]:
x_train = train_features/255.0
x_test = test_features/255.0

In [16]:
from sklearn.preprocessing import LabelEncoder

In [17]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [19]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [20]:
def convolutional_block(X, f, filters, stage, block, s=2):

    # Retrieve Filters
    F1, F2, F3 = filters

    # Define name basis
    conv_base = 'res' + str(stage) + block + '_branch'
    bn_base = 'bn' + str(stage) + block + '_branch'

    # Save the input value for the shortcut path
    X_shortcut = X

    ##### MAIN PATH #####

    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s), name=conv_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same', name=conv_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides=(1, 1), name=conv_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_base + '2c')(X)

    ##### SHORTCUT PATH #### (to match dimensions)
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), name=conv_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_base + '1')(X_shortcut)

    # Add shortcut value to the main path, and pass through RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [21]:
def identity_block(X, f, filters, stage, block):

    # Retrieve Filters
    F1, F2, F3 = filters

    # Define name basis
    conv_base = 'res' + str(stage) + block + '_branch'
    bn_base = 'bn' + str(stage) + block + '_branch'

    # Save the input value for the shortcut path
    X_shortcut = X

    ##### MAIN PATH #####

    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(1, 1), name=conv_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same', name=conv_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides=(1, 1), name=conv_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_base + '2c')(X)

    # Add shortcut value to the main path, and pass through RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

In [ ]:
def ResNet50(input_shape=(48, 48, 1), classes=7):


    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # Average Pooling
    X = AveragePooling2D(pool_size=(2, 2), name="avg_pool")(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model
ResNet50(input_shape = (48, 48, 1), classes = 7)

<Functional name=ResNet50, built=True>

In [23]:
early_stopping = EarlyStopping(
    monitor='val_loss',       # Theo dõi chỉ số 'val_loss' (loss trên tập validation)
    patience=5,               # Số epoch không cải thiện trước khi dừng
    restore_best_weights=True,  # Phục hồi trọng số tốt nhất của mô hình
    min_delta=0.001           # Mức cải thiện tối thiểu (ở đây là 0.001)
)

In [24]:
model = ResNet50(input_shape=(48, 48, 1), classes=7)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'] 
)

In [25]:
model.fit(
    x=x_train, 
    y=y_train, 
    batch_size=128, 
    epochs=100, 
    validation_data=(x_test, y_test), 
    callbacks=[early_stopping]  # Thêm Early Stopping vào đây
)

Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 429s 2s/step - accuracy: 0.2744 - loss: 2.5231 - val_accuracy: 0.1615 - val_loss: 2.0461
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 384s 2s/step - accuracy: 0.3954 - loss: 1.6419 - val_accuracy: 0.2583 - val_loss: 235157.8438
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 385s 2s/step - accuracy: 0.2565 - loss: 1.9539 - val_accuracy: 0.3239 - val_loss: 2.0380
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 377s 2s/step - accuracy: 0.3721 - loss: 1.6937 - val_accuracy: 0.3856 - val_loss: 1.5931
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 369s 2s/step - accuracy: 0.4235 - loss: 1.5901 - val_accuracy: 0.3017 - val_loss: 4.4860
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 382s 2s/step - accuracy: 0.4554 - loss: 1.4391 - val_accuracy: 0.4478 - val_loss: 1.5051
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 366s 2s/step - accuracy: 0.4846 - loss: 1.3739 - val_accuracy: 0.4567 - val_loss: 1.9891
Epoch 8/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 374s 2s/step - accuracy: 0.5092 - loss: 1.30

In [26]:
pip install pydot  

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\khang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [30]:
from tensorflow.keras.utils import plot_model 
print(model.summary())

# Visualize the model architecture
plot_model(
    model, 
    to_file='model_architecture.png',  # Save the model diagram as an image
    show_shapes=True,                  # Display input/output shapes
    show_layer_names=True              # Display layer names
)

Model: "ResNet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 48, 48, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 54, 54, 1) │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1 (Conv2D)      │ (None, 24, 24,    │      3,200 │ zero_padding2d_1… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_conv1            │ (None, 24, 24,    │        256 │ conv1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_49       │ (None, 24, 24,    │          0 │ bn_conv1[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 11, 11,    │          0 │ activation_49[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2a_branch2a      │ (None, 11, 11,    │      4,160 │ max_pooling2d_1[… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2a_branch2a       │ (None, 11, 11,    │        256 │ res2a_branch2a[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_50       │ (None, 11, 11,    │          0 │ bn2a_branch2a[0]… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2a_branch2b      │ (None, 11, 11,    │     36,928 │ activation_50[0]… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2a_branch2b       │ (None, 11, 11,    │        256 │ res2a_branch2b[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_51       │ (None, 11, 11,    │          0 │ bn2a_branch2b[0]… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2a_branch2c      │ (None, 11, 11,    │     16,640 │ activation_51[0]… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ res2a_branch1       │ (None, 11, 11,    │     16,640 │ max_pooling2d_1[… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2a_branch2c       │ (None, 11, 11,    │      1,024 │ res2a_branch2c[0… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn2a_branch1        │ (None, 11, 11,    │      1,024 │ res2a_branch1[0]… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_16 (Add)        │ (None, 11, 11,    │          0 │ bn2a_branch2c[0]

 Total params: 70,681,111 (269.63 MB)

 Trainable params: 23,542,663 (89.81 MB)

 Non-trainable params: 53,120 (207.50 KB)

 Optimizer params: 47,085,328 (179.62 MB)

None
You must install graphviz (see instructions at https://graphviz.gitlab.io/download/) for `plot_model` to work.


In [31]:
file_path = 'C:/Users/khang/OneDrive/Desktop/Machine_Learning'
# Lưu model bằng file json
model_json = model.to_json()
with open(os.path.join(file_path, "emotiondetector_resnet50.json"), 'w') as json_file:
    json_file.write(model_json)

# Lưu trọng số của model
model.save(os.path.join(file_path, "emotiondetector_resnet50.h5"))